In [15]:
import os
import re
import time
import pandas as pd
from openai import RateLimitError
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores.faiss import FAISS
from langchain_core.documents import Document
from langchain_text_splitters import CharacterTextSplitter
from langchain_community.document_loaders import TextLoader

In [4]:
from dotenv import load_dotenv,find_dotenv
load_dotenv(find_dotenv())
OpenAI_key = os.environ.get("OPENAI_API_KEY")

In [16]:
loader = TextLoader("state_of_the_union.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
docs = text_splitter.split_documents(documents)
embeddings = OpenAIEmbeddings()
db = FAISS.from_documents(docs, embeddings)

In [18]:
# Save the vector store to a file for future use
save_path = f"state_of_the_union_faiss_index.faiss"
db.save_local(save_path)

In [19]:
import os
import re
from cachetools import TTLCache
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain_community.callbacks import get_openai_callback
from langchain_core.messages import HumanMessage, AIMessage
from langchain_core.prompts import PromptTemplate
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings, ChatOpenAI

In [38]:

cache = TTLCache(maxsize=100, ttl=3600)
embeddings = OpenAIEmbeddings()


def load_faiss_index(file_path):
    global cache
    cached_index = cache.get("faiss_index")
    if cached_index:
        return cached_index
    vectorstore = FAISS.load_local(file_path, embeddings,allow_dangerous_deserialization=True)
    cache["faiss_index"] = vectorstore
    return vectorstore

In [67]:
vectorstore = load_faiss_index('state_of_the_union_faiss_index.faiss')
retriever = vectorstore.as_retriever(k=4)
temperature = 0.2  # Fine-tune for more creative or conservative responses
model_name = 'gpt-3.5-turbo-1106'

In [41]:
system_template = """
Here's what I know:

Context: {context}
Conversation so far: {chat_history}
Your question: {question}
"""

In [43]:
prompt = ChatPromptTemplate.from_messages(
    messages=[
        SystemMessagePromptTemplate(
            prompt=PromptTemplate(input_variables=["context", "chat_history", "question"],
                                    template=system_template)),
        HumanMessagePromptTemplate.from_template("{question}")
    ]
)

In [45]:
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True, output_key='answer')

In [48]:
qa_chain = ConversationalRetrievalChain.from_llm(llm=ChatOpenAI(model=model_name, temperature=temperature),
                                                  retriever=retriever, 
                                                  verbose=False,
                                                              memory=memory,
                                                              combine_docs_chain_kwargs={"prompt": prompt})

In [57]:
def optimize_user_input( text):
    text = re.sub(r'[^\w\s]', '', text) 
    text = re.sub(r'\s+', ' ', text).strip() 
    return text

In [58]:
def predict( query):
    optimized_query = optimize_user_input(query)
    cache_key = f"response_{optimized_query}"

    cached_response = cache.get(cache_key)
    if cached_response:
        return cached_response

    user_message = HumanMessage(content=optimized_query)
    memory.chat_memory.add_message(user_message)
    response = qa_chain.invoke({"question": optimized_query})
    ai_response = AIMessage(content=response['answer'])
    memory.chat_memory.add_message(ai_response)

    cache[cache_key] = response['answer']
    return response['answer']

In [64]:
user_input = 'What was  Vladimir Putins  action?'

In [65]:
predict(query=user_input)

"Based on the context provided, Vladimir Putin's recent actions regarding Ukraine are the focus. Some of his specific actions include the premeditated and unprovoked attack on Ukraine, the rejection of repeated efforts at diplomacy, and the belief that the West and NATO wouldn't respond to his aggression. Additionally, the speech mentions that the free world is holding him accountable through economic sanctions, cutting off Russia's largest banks from the international financial system, and preventing Russia's central bank from defending the Russian Ruble. These actions have led to Russia being isolated from the world and facing significant economic consequences. If you have specific questions about any of these actions or policies, feel free to ask for more information."

In [66]:
qa_chain.invoke({"question": user_input})

{'question': 'What was  Vladimir Putins  action?',
 'chat_history': [HumanMessage(content='What did Vladimir Putin said'),
  HumanMessage(content='What did Vladimir Putin said'),
  AIMessage(content="I'm sorry, but I don't have direct access to Vladimir Putin's statements. However, based on the context provided, it seems that Vladimir Putin's actions and decisions are being discussed, rather than his specific statements. If you have any specific questions about his actions or policies, feel free to ask and I'll do my best to provide information based on the context given."),
  AIMessage(content="I'm sorry, but I don't have direct access to Vladimir Putin's statements. However, based on the context provided, it seems that Vladimir Putin's actions and decisions are being discussed, rather than his specific statements. If you have any specific questions about his actions or policies, feel free to ask and I'll do my best to provide information based on the context given."),
  HumanMessage(

In [68]:

user_input = 'Tell me about Putin’s latest attack on Ukraine?'
qa_chain.invoke({"question": user_input})

{'question': 'Tell me about Putin’s latest attack on Ukraine?',
 'chat_history': [HumanMessage(content='What did Vladimir Putin said'),
  HumanMessage(content='What did Vladimir Putin said'),
  AIMessage(content="I'm sorry, but I don't have direct access to Vladimir Putin's statements. However, based on the context provided, it seems that Vladimir Putin's actions and decisions are being discussed, rather than his specific statements. If you have any specific questions about his actions or policies, feel free to ask and I'll do my best to provide information based on the context given."),
  AIMessage(content="I'm sorry, but I don't have direct access to Vladimir Putin's statements. However, based on the context provided, it seems that Vladimir Putin's actions and decisions are being discussed, rather than his specific statements. If you have any specific questions about his actions or policies, feel free to ask and I'll do my best to provide information based on the context given."),
  